In [255]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='west0', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0...


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39589)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:26


In [254]:
sphynx.stop()

In [27]:
# user 지역
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

# user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
# user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left")

[기준] 위 구매자들 중 Battlegrounds Plus 상품은 구매하지 않은 Basic 멤버쉽 유저들: 95,359명  (user_list)  

1/12 ~ 1/25 2주간 이들의 G-Coin 사용  
-> 에서 밀수품 쿠폰 사용이 가장 많았으므로, 이들의 STARTER PACKAGE구매 시간을 가지고 CONTRABAND 오픈 시간과 비교하여 분석

In [271]:
starter_buyer_cash_buy = pd.read_csv('./data/starter_buyer_cash_buy.csv')
# starter package 구매자들 중 plus 안산고 다른 상품 구매한 유저들
starter_and_plus_buyer_list = starter_buyer_cash_buy[starter_buyer_cash_buy["product_id"] == "gcoinbundledesc.1015"]["account_id"].unique().tolist()
no_plus_cash_buy = starter_buyer_cash_buy[~starter_buyer_cash_buy["account_id"].isin(starter_and_plus_buyer_list)]
user_list = no_plus_cash_buy["account_id"].unique().tolist()

In [272]:
len(user_list)

95359

In [273]:
# starter package 구매 시간
sp_buy_time = no_plus_cash_buy[no_plus_cash_buy.product_id == "gcoinbundledesc.1016"][["date", "account_id", "time"]]

In [25]:
no_plus_cash_buy[no_plus_cash_buy.product_id == "gcoinbundledesc.1016"][["date", "account_id", "time"]].date.unique()

array(['2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15',
       '2022-01-16', '2022-01-17', '2022-01-18'], dtype=object)

In [274]:
# 이들의 gcoin use
sp_gcoin_df = pd.read_csv("./data/starter_buyer_gcoin.csv")

In [275]:
sp_gcoin_df.shape

(233014, 35)

In [52]:
sp_gcoin_df[sp_gcoin_df.event_name.str.contains("wsus")].head()

,date,platform,account_id,product_id,free_use,free_balance,paid_use,paid_balance,price,qty,...,event_type,event_name,tags,part,tier,price_,start_time,end_time,catalog_version,update_time
0,2022-01-12,KAKAO,account.0b22bc023c1644a39307742e56ccb255,itemdesc.14100012,200,100,0,0,200,1,...,wsus,202112_wsus_progressive,NaN,crate,1,200.0,2021-12-01 02:00:00,2022-03-31 02:00:00,715,2022-04-01 10:29:23
2,2022-01-12,KAKAO,account.1b3eaa8d4bba4ba796eea4965b899cc2,itemdesc.14100013,200,200,0,0,200,1,...,wsus,202201_wsus,NaN,crate,1,200.0,2022-01-11 02:00:00,2022-04-30 02:00:00,715,2022-04-01 10:29:23
3,2022-01-12,KAKAO,account.1eb267a576324df381c0bbde475d869b,itemdesc.14100013,200,400,0,0,200,1,...,wsus,202201_wsus,NaN,crate,1,200.0,2022-01-11 02:00:00,2022-04-30 02:00:00,715,2022-04-01 10:29:23
5,2022-01-12,KAKAO,account.2197b1fe241c426c89dc918b6cef45db,itemdesc.14100013,200,100,0,0,200,1,...,wsus,202201_wsus,NaN,crate,1,200.0,2022-01-11 02:00:00,2022-04-30 02:00:00,715,2022-04-01 10:29:23
6,2022-01-12,KAKAO,account.2626ecc1eeac4c5c8cfafc13ab7af4c2,itemdesc.14100013,200,300,0,0,200,1,...,wsus,202201_wsus,NaN,crate,1,200.0,2022-01-11 02:00:00,2022-04-30 02:00:00,715,2022-04-01 10:29:23


In [282]:
wsus_gcoin_user = sp_gcoin_df[sp_gcoin_df.event_name.str.contains("wsus")].account_id.unique().tolist()

In [277]:
len(wsus_gcoin_user)

40469

In [278]:
wsus = load_schema.lobby(spark, "pc", env="live", log_type="ContrabandCrateOpened", start_date="2022-01-12", end_date="2022-01-25")

In [279]:
wsus = wsus.where(col("AccountId").isin(user_list))

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:42663)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:42663)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [315]:
wsus.groupBy("CurrencyUsedType").agg(countDistinct("AccountId")).show()

+----------------+-------------------------+
|CurrencyUsedType|count(DISTINCT AccountId)|
+----------------+-------------------------+
|            cash|                    40469|
|contrabandcoupon|                    89282|
+----------------+-------------------------+



In [178]:
wsus_data = wsus.toPandas()

In [179]:
wsus_data = pd.merge(wsus_data, sp_buy_time, left_on="AccountId", right_on="account_id")

In [183]:
# wsus_data[["Time", "time"]] = wsus_data[["Time", "time"]].apply(pd.to_datetime)
wsus_tmp_data = wsus_data[(wsus_data["Time"] > wsus_data["time"]) & (wsus_data["CurrencyUsedType"] == "contrabandcoupon")]

In [184]:
wsus_tmp_data["total"] = wsus_tmp_data["CurrencyUsedAmounts"] * wsus_tmp_data["OpenAmount"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [187]:
wsus_tmp_data = wsus_tmp_data.groupby("AccountId").filter(lambda x: x["total"].sum() > 50)

In [191]:
wsus_tmp_data[wsus_tmp_data.AccountId.isin(wsus_gcoin_user)].AccountId.nunique()

34696

In [297]:
wsus_gcoin_user = wsus_tmp_data[wsus_tmp_data.AccountId.isin(wsus_gcoin_user)].AccountId.unique().tolist()

In [316]:
wsus_gcoin_big_df = sp_gcoin_df[(sp_gcoin_df.event_name.str.contains("wsus")) & (sp_gcoin_df.account_id.isin(user_list))]

In [300]:
wsus_gcoin_df = sp_gcoin_df[(sp_gcoin_df.event_name.str.contains("wsus")) & (sp_gcoin_df.account_id.isin(wsus_gcoin_user))]

In [317]:
wsus_gcoin_big_df.account_id.nunique()

40469

In [313]:
wsus_gcoin_df.account_id.nunique()

34696

In [318]:
wsus_gcoin_big_df[wsus_gcoin_big_df.paid_use == 0].account_id.nunique()

39136

In [304]:
wsus_gcoin_df[wsus_gcoin_df.paid_use == 0].account_id.nunique()

33612

In [319]:
wsus_gcoin_big_df[wsus_gcoin_big_df.paid_use == 0].groupby("price_")["account_id"].count()

price_
200.0     118163
1800.0       235
Name: account_id, dtype: int64

In [309]:
wsus_gcoin_df[wsus_gcoin_df.paid_use == 0].groupby("price_")["account_id"].count()

price_
200.0     105870
1800.0       227
Name: account_id, dtype: int64

In [320]:
wsus_gcoin_big_df[wsus_gcoin_big_df.free_use == 0].account_id.nunique()

4492

In [312]:
wsus_gcoin_df[wsus_gcoin_df.free_use == 0].account_id.nunique()

3956

In [321]:
wsus_gcoin_big_df[wsus_gcoin_big_df.paid_use > 0].account_id.nunique()

8477

In [305]:
wsus_gcoin_df[wsus_gcoin_df.paid_use > 0].account_id.nunique()

7657

In [310]:
wsus_gcoin_df[wsus_gcoin_df.paid_use > 0].groupby("price_")["account_id"].count()

price_
200.0     16004
1800.0     3637
Name: account_id, dtype: int64

In [311]:
wsus_gcoin_df[wsus_gcoin_df.paid_use > 0].groupby("price_")[["free_use", "paid_use"]].describe()

free_use                                                         \
          count        mean         std  min  25%   50%    75%     max   
price_                                                                   
200.0   16004.0   43.100475   55.579834  0.0  0.0   0.0  100.0   190.0   
1800.0   3637.0  268.979929  420.426768  0.0  0.0  10.0  400.0  1790.0   

       paid_use                                                                 
          count         mean         std   min     25%     50%     75%     max  
price_                                                                          
200.0   16004.0   156.899525   55.579834  10.0   100.0   200.0   200.0   200.0  
1800.0   3637.0  1531.020071  420.426768  10.0  1400.0  1790.0  1800.0  1800.0

In [322]:
free_big_list = wsus_gcoin_big_df[wsus_gcoin_big_df.paid_use == 0].account_id.unique().tolist()
paid_big_list = wsus_gcoin_big_df[wsus_gcoin_big_df.paid_use > 0].account_id.unique().tolist()
free_and_paid_big = [i for i in free_big_list if i in paid_big_list]

In [306]:
free_list = wsus_gcoin_df[wsus_gcoin_df.paid_use == 0].account_id.unique().tolist()
paid_list = wsus_gcoin_df[wsus_gcoin_df.paid_use > 0].account_id.unique().tolist()
free_and_paid = [i for i in free_list if i in paid_list]

In [323]:
len(free_and_paid_big)

7144

In [307]:
len(free_and_paid)

6573

In [308]:
wsus_gcoin_df[wsus_gcoin_df.account_id.isin(free_and_paid)].describe()

,free_use,free_balance,paid_use,paid_balance,price,qty,transaction_id,is_salesid_exist,sales_id_,sales_id,tier,price_,catalog_version
count,41090.000000,41090.000000,41090.000000,41090.000000,41090.000000,41090.0,4.109000e+04,41090.0,41090.000000,41090.000000,41090.0,41090.000000,41090.0
mean,147.127525,184.649793,160.382818,201.298369,307.510343,1.0,1.482737e+18,1.0,1840.738477,1840.738477,1.0,307.510343,715.0
std,140.430498,353.230710,398.881967,1209.722849,400.577059,0.0,1.243197e+15,0.0,68.111920,68.111920,0.0,400.577059,0.0
min,0.000000,0.000000,0.000000,0.000000,200.000000,1.0,1.480614e+18,1.0,1729.000000,1729.000000,1.0,200.000000,715.0
25%,30.000000,0.000000,0.000000,0.000000,200.000000,1.0,1.481807e+18,1.0,1778.000000,1778.000000,1.0,200.000000,715.0
50%,200.000000,100.000000,0.000000,0.000000,200.000000,1.0,1.482373e+18,1.0,1889.000000,1889.000000,1.0,200.000000,715.0
75%,200.000000,260.000000,200.000000,10.000000,200.000000,1.0,1.483369e+18,1.0,1889.000000,1889.000000,1.0,200.000000,715.0
max,1800.000000,8100.000000,1800.000000,58510.000000,1800.000000,1.0,1.486097e+18,1.0,1890.000000,1890.000000,1.0,1800.000000,715.0


In [75]:
wsus_first_time = wsus.groupBy("AccountId", "CurrencyUsedType").agg(min("Time").alias("first_time")).toPandas()

In [78]:
wsus_first_time.head()

,AccountId,CurrencyUsedType,first_time
0,account.0058bc6f3d154094bb374fe0a890fec9,contrabandcoupon,2022-01-12T18:19:29.7890812Z
1,account.00851dfd9dac4b74b90f7a88fa6a9276,contrabandcoupon,2022-01-12T12:38:23.5393988Z
2,account.013651c7fb0a4bc9ad3a8adcf19cd705,contrabandcoupon,2022-01-12T15:17:28.1996606Z
3,account.01888fbe33f040a39e9b89185163c128,cash,2022-01-16T10:42:02.9721671Z
4,account.026d8ed280cf409691b374f0b5124fe4,contrabandcoupon,2022-01-15T20:16:27.5864617Z


In [80]:
sp_buy_time.shape

(95359, 3)

In [81]:
wsus_first_time.shape

(129751, 3)

In [84]:
wsus_first_time.groupby("CurrencyUsedType")["AccountId"].count()

CurrencyUsedType
cash                40469
contrabandcoupon    89282
Name: AccountId, dtype: int64

In [79]:
df = pd.merge(sp_buy_time, wsus_first_time, left_on="account_id", right_on="AccountId")
df[(df["CurrencyUsedType"]) & (df['time'] < df['first_time'])]

,date,account_id,time,AccountId,CurrencyUsedType,first_time
0,2022-01-12,account.05461471521746b2bec07d45a9387392,2022-01-12T09:18:38.0775884Z,account.05461471521746b2bec07d45a9387392,contrabandcoupon,2022-01-12T06:06:45.0051915Z
1,2022-01-12,account.0ff87c577eaa4b0d868ce4423afefb4d,2022-01-12T15:09:59.0561961Z,account.0ff87c577eaa4b0d868ce4423afefb4d,contrabandcoupon,2022-01-12T15:22:12.6207823Z
2,2022-01-12,account.0ff87c577eaa4b0d868ce4423afefb4d,2022-01-12T15:09:59.0561961Z,account.0ff87c577eaa4b0d868ce4423afefb4d,cash,2022-01-13T14:45:39.1738564Z
3,2022-01-12,account.114799bf4a6e4badbe131115ee804938,2022-01-12T06:40:40.1135256Z,account.114799bf4a6e4badbe131115ee804938,contrabandcoupon,2022-01-12T06:41:34.1354619Z
4,2022-01-12,account.1444ced69f32486996f1c55a3eb3b75d,2022-01-12T08:40:47.3500631Z,account.1444ced69f32486996f1c55a3eb3b75d,contrabandcoupon,2022-01-12T08:32:27.6909218Z
5,2022-01-12,account.1d09e4675201406db977b961372dfe99,2022-01-12T20:54:53.4626264Z,account.1d09e4675201406db977b961372dfe99,contrabandcoupon,2022-01-12T21:03:14.824813Z
6,2022-01-12,account.1e2bc5c9478c4a18a46e10de06c8be05,2022-01-12T09:27:48.9955141Z,account.1e2bc5c9478c4a18a46e10de06c8be05,contrabandcoupon,2022-01-12T09:28:51.2426711Z
7,2022-01-12,account.1ef8dca4a38140af90e191e99c3bdfe8,2022-01-12T08:26:00.956671Z,account.1ef8dca4a38140af90e191e99c3bdfe8,cash,2022-01-12T10:49:08.0524047Z
8,2022-01-12,account.1ef8dca4a38140af90e191e99c3bdfe8,2022-01-12T08:26:00.956671Z,account.1ef8dca4a38140af90e191e99c3bdfe8,contrabandcoupon,2022-01-12T08:22:03.408848Z
9,2022-01-12,account.21c37c1d25eb4797be56afa321c42a3d,2022-01-12T16:07:04.3984492Z,account.21c37c1d25eb4797be56afa321c42a3d,contrabandcoupon,2022-01-12T16:08:36.6402812Z


In [280]:
wsus_df = wsus.groupBy("AccountId","date","CurrencyUsedType","CurrencyUsedAmounts").agg(sum("OpenAmount").alias("open_cnt")).toPandas()

In [285]:
wsus_df.head()

,AccountId,date,CurrencyUsedType,CurrencyUsedAmounts,open_cnt,total
0,account.9ead96b27947473cb36b70689a9f56a0,2022-01-12,cash,200,4,800
1,account.3ee2f86b9c874eae9fb7ce2a67e4f50c,2022-01-12,cash,200,2,400
2,account.f0e64e84949d463d99e7c0cc99d06f7b,2022-01-12,contrabandcoupon,10,5,50
3,account.b4c13ac721cc4549970432153063eed7,2022-01-12,contrabandcoupon,10,6,60
4,account.45a1a3e6ee9d45a69800793c0e1de070,2022-01-12,contrabandcoupon,10,9,90


In [281]:
wsus_df.AccountId.nunique()

89623

In [283]:
wsus_df["total"] = wsus_df["CurrencyUsedAmounts"] * wsus_df["open_cnt"]

In [284]:
wsus_df[(wsus_df["AccountId"].isin(wsus_gcoin_user)) & (wsus_df["CurrencyUsedType"] == "contrabandcoupon")].describe()

,CurrencyUsedAmounts,open_cnt,total
count,112075.0,112075.000000,112075.000000
mean,10.0,3.585278,35.852777
std,0.0,2.687407,26.874066
min,10.0,1.000000,10.000000
25%,10.0,1.000000,10.000000
50%,10.0,3.000000,30.000000
75%,10.0,6.000000,60.000000
max,10.0,20.000000,200.000000


In [170]:
wsus_df[(wsus_df["AccountId"].isin(wsus_gcoin_user))].AccountId.nunique()

40469

In [169]:
wsus_df[(wsus_df["AccountId"].isin(wsus_gcoin_user))].groupby("CurrencyUsedType")["AccountId"].nunique()

CurrencyUsedType
cash                40469
contrabandcoupon    40128
Name: AccountId, dtype: int64

In [168]:
wsus_df[(wsus_df["AccountId"].isin(wsus_gcoin_user))].groupby("CurrencyUsedType").describe()

CurrencyUsedAmounts                                        \
                               count        mean         std    min    25%   
CurrencyUsedType                                                             
cash                         63693.0  256.219679  294.605153  200.0  200.0   
contrabandcoupon            112075.0   10.000000    0.000000   10.0   10.0   

                                        open_cnt            ...              \
                    50%    75%     max     count      mean  ...  75%    max   
CurrencyUsedType                                            ...               
cash              200.0  200.0  1800.0   63693.0  2.773853  ...  3.0  500.0   
contrabandcoupon   10.0   10.0    10.0  112075.0  3.585278  ...  6.0   20.0   

                     total                                                  \
                     count         mean           std    min    25%    50%   
CurrencyUsedType                                                             
cash               63693.0  1587.725496  10381.994081  200.0  200.0  400.0   
contrabandcoupon  112075.0    35.852777     26.874066   10.0   10.0   30.0   

                                   
                    75%       max  
CurrencyUsedType                   
cash              600.0  900000.0  
contrabandcoupon   60.0     200.0  

[2 rows x 24 columns]

In [288]:
target_user = wsus_df[wsus_df["CurrencyUsedType"] == "contrabandcoupon"].groupby("AccountId", as_index=False).filter(lambda x: x["total"].sum() >= 50).AccountId.unique().tolist()

In [289]:
len(target_user)

88189

In [290]:
target_wsus_gcoin_user = [i for i in wsus_gcoin_user if i in target_user]

In [291]:
len(target_wsus_gcoin_user) 

40012

In [294]:
wsus_cash_user = wsus_df[wsus_df.CurrencyUsedType == "cash"].AccountId.unique()
wsus_coupon_user = wsus_df[wsus_df.CurrencyUsedType == "contrabandcoupon"].AccountId.unique()

In [295]:
wsus_both_user = [i for i in wsus_coupon_user if i in wsus_cash_user]

In [296]:
print("WSUS cash user:", len(wsus_cash_user))
print("WSUS coupon user:", len(wsus_coupon_user))
print("WSUS cash and coupon:", len(wsus_both_user))

WSUS cash user: 40469
WSUS coupon user: 89282
WSUS cash and coupon: 40128


In [85]:
no_coupon_user = [i for i in wsus_cash_user if i not in wsus_both_user]

In [138]:
wsus_tmp = wsus.where((col("AccountId").isin(wsus_both_user))).groupBy("AccountId").agg(min(when(col("CurrencyUsedType")=="contrabandcoupon", col("Time"))).alias("first_time_coupon_open"), max(when(col("CurrencyUsedType")=="contrabandcoupon", col("Time"))).alias("last_time_coupon_open"), min(when(col("CurrencyUsedType")=="cash", col("Time"))).alias("first_time_cash_open"))

In [140]:
wsus_tmp_df = wsus_tmp.toPandas()

In [141]:
wsus_tmp_df[['first_time_cash_open', 'first_time_coupon_open', 'last_time_coupon_open']] = wsus_tmp_df[['first_time_cash_open', 'first_time_coupon_open', 'last_time_coupon_open']].apply(pd.to_datetime)
# wsus_tmp_df["timediff"] = (wsus_tmp_df.first_time_cash_open - wsus_tmp_df.last_time_coupon_open).astype('timedelta64[h]')

In [142]:
wsus_tmp_df.head()

,AccountId,first_time_coupon_open,last_time_coupon_open,first_time_cash_open
0,account.026079fd232044679f486155c23d67ac,2022-01-12 16:44:58.113901200+00:00,2022-01-12 16:51:02.689975500+00:00,2022-01-12 16:54:27.305796200+00:00
1,account.02726b721c6240b48db71e8032068e89,2022-01-12 13:02:56.054275300+00:00,2022-01-23 06:37:00.395935200+00:00,2022-01-12 13:39:16.038315300+00:00
2,account.05d9114ec8f6496d9d4edfb50e4782da,2022-01-14 19:15:36.856913400+00:00,2022-01-14 19:17:41.822853400+00:00,2022-01-12 22:18:37.803175200+00:00
3,account.067e5f9972e64e50a74a248284208c58,2022-01-12 17:34:16.402742700+00:00,2022-01-23 10:52:06.843930200+00:00,2022-01-20 12:31:43.012566800+00:00
4,account.074e8078700a49a189b5e74ed5e72e82,2022-01-15 05:09:28.096051100+00:00,2022-01-22 11:53:27.250666700+00:00,2022-01-13 09:00:09.274628100+00:00


In [127]:
wsus_tmp_df[wsus_tmp_df.timediff < 0].shape

(33391, 4)

In [128]:
wsus_tmp_df[wsus_tmp_df.timediff >= 0].shape

(6737, 4)

In [143]:
sp_buy_time.time = sp_buy_time.time.apply(pd.to_datetime)
df = pd.merge(sp_buy_time, wsus_tmp_df, left_on="account_id", right_on="AccountId")

In [144]:
df.head()

,date,account_id,time,AccountId,first_time_coupon_open,last_time_coupon_open,first_time_cash_open
0,2022-01-12,account.0ff87c577eaa4b0d868ce4423afefb4d,2022-01-12 15:09:59.056196100+00:00,account.0ff87c577eaa4b0d868ce4423afefb4d,2022-01-12 15:22:12.620782300+00:00,2022-01-13 14:49:40.683186900+00:00,2022-01-13 14:45:39.173856400+00:00
1,2022-01-12,account.1ef8dca4a38140af90e191e99c3bdfe8,2022-01-12 08:26:00.956671+00:00,account.1ef8dca4a38140af90e191e99c3bdfe8,2022-01-12 08:22:03.408848+00:00,2022-01-22 08:43:12.323616+00:00,2022-01-12 10:49:08.052404700+00:00
2,2022-01-12,account.30358205fbed411aae53c1e0940efa0e,2022-01-12 05:42:22.650966+00:00,account.30358205fbed411aae53c1e0940efa0e,2022-01-12 05:43:01.059836+00:00,2022-01-12 05:44:52.440807+00:00,2022-01-12 05:45:34.664817300+00:00
3,2022-01-12,account.3a0cc85d50d44ff78f65e44495d4b433,2022-01-12 10:25:47.401540500+00:00,account.3a0cc85d50d44ff78f65e44495d4b433,2022-01-12 10:26:27.524380800+00:00,2022-01-14 09:00:14.405732600+00:00,2022-01-12 10:26:55.200912300+00:00
4,2022-01-12,account.586f7be4c23d4fbb99692e0ee7074182,2022-01-12 14:40:56.932804600+00:00,account.586f7be4c23d4fbb99692e0ee7074182,2022-01-12 14:42:33.073697400+00:00,2022-01-13 14:34:34.929851+00:00,2022-01-12 14:41:31.847541300+00:00


In [173]:
df[df["account_id"].isin(wsus_gcoin_user)].account_id.nunique()

40128

In [175]:
df[(df["account_id"].isin(wsus_gcoin_user)) & (df.time < df.first_time_cash_open)].shape

(22597, 7)

In [65]:
wsus_df[wsus_df.CurrencyUsedType == "contrabandcoupon"].CurrencyUsedAmounts.unique()

array([10])

In [326]:
sp = load_data_mart("pc", "2022-01-12", "2022-01-18", "cash_mtx").where((col("product_id") == "gcoinbundledesc.1016") & (~col("account_id").isin(user_list)))

In [330]:
legacy_user = sp.select("account_id").toPandas()["account_id"].unique().tolist()
legacy_wsus_gcoin = load_data_mart("pc", "2022-01-12", "2022-01-25", "gcoin_use").where((col("account_id").isin(legacy_user)) & (col("event_name").like("%wsus%")))

In [335]:
len(legacy_user)

872930

In [338]:
legacy_wsus_gcoin.select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                    613267|
+--------------------------+



In [332]:
legacy_wsus_gcoin.where(col("paid_use") == 0).select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                    546911|
+--------------------------+



In [336]:
546911/872930*100

62.65233180209181

In [333]:
legacy_wsus_gcoin.where(col("paid_use") > 0).select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                    252041|
+--------------------------+



In [337]:
252041/872930*100

28.872990961474578

In [331]:
membership = load_membership("pubg", "pc", "2022-01-25", "2022-01-25").where(col("account_id").isin(legacy_user))
membership.groupBy("membership").agg(countDistinct("account_id")).show()

+----------+--------------------------+
|membership|count(DISTINCT account_id)|
+----------+--------------------------+
|      plus|                     11760|
|     basic|                       141|
|    legacy|                    861029|
+----------+--------------------------+



In [ ]:
legacy_wsus = load_schema.lobby(spark, "pc", env="live", log_type="ContrabandCrateOpened", start_date="2022-01-12", end_date="2022-01-25").where(col("AccountId").isin(legacy_user))

In [ ]:
legacy_wsus.groupBy("CurrencyUsedType").agg(countDistinct("AccountId")).show()

첫 cash상품 구매 = starter package?

In [259]:
cash_mtx = load_data_mart("pc", "2022-01-12", "2022-02-08", "cash_mtx")

In [261]:
cash_mtx = cash_mtx.where(col("product_id") == "gcoinbundledesc.1016")

In [260]:
pu_master = load_data_mart("pc", "2022-02-08", "2022-02-08", "pu_master").drop("date")

In [262]:
membership = load_membership("pubg", "pc", "2022-02-08", "2022-02-08")
membership = membership.select("account_id", "membership").distinct()

In [263]:
cash_pu = cash_mtx.join(pu_master, "account_id")

In [264]:
cash_pu = cash_pu.join(membership, "account_id")

In [267]:
tmp = cash_pu.where((col("membership") != "legacy") & (col("date") == "2022-01-12") & (col("first_mtx_date") != "2022-01-12"))

In [270]:
tmp.select("date", "first_mtx_date", "membership").show(truncate=False)

+----------+--------------+----------+
|date      |first_mtx_date|membership|
+----------+--------------+----------+
|2022-01-12|2021-11-27    |basic     |
|2022-01-12|2021-07-08    |basic     |
|2022-01-12|2018-11-01    |basic     |
|2022-01-12|2021-10-14    |basic     |
|2022-01-12|2018-08-23    |plus      |
|2022-01-12|2020-12-21    |basic     |
|2022-01-12|2020-11-20    |plus      |
|2022-01-12|2020-10-09    |basic     |
|2022-01-12|2019-02-06    |basic     |
|2022-01-12|2019-01-31    |plus      |
|2022-01-12|2021-10-16    |basic     |
|2022-01-12|2018-06-28    |basic     |
|2022-01-12|2018-05-19    |basic     |
|2022-01-12|2019-09-02    |basic     |
|2022-01-12|2018-06-06    |basic     |
|2022-01-12|2019-02-19    |plus      |
|2022-01-12|2019-01-14    |plus      |
|2022-01-12|2019-09-19    |basic     |
|2022-01-12|2018-12-20    |basic     |
|2022-01-12|2021-12-12    |basic     |
+----------+--------------+----------+
only showing top 20 rows



In [251]:
cash_pu.withColumn("legacy", when(col("membership") == "legacy", lit("legacy")).otherwise(lit("non_legacy"))).groupBy(["date", "legacy"]).agg(countDistinct("account_id").alias("pu"), countDistinct(when(col("date") == col("first_mtx_date"), col("account_id"))).alias("npu")).withColumn("npur", (col("npu")/col("pu"))).orderBy("date").toPandas().pivot_table(values=["pu", "npu", "npur"], columns="legacy", index="date")

npu                 npur                 pu           
legacy     legacy non_legacy    legacy non_legacy  legacy non_legacy
date                                                                
2022-01-12  57331      19274  0.140811   0.946428  407148      20365
2022-01-13  31860      18884  0.206699   0.951048  154137      19856
2022-01-14  22406      16581  0.233493   0.952767   95960      17403
2022-01-15  24023      17854  0.245345   0.945406   97915      18885
2022-01-16  13516      12123  0.265098   0.941739   50985      12873
2022-01-17   9015       9145  0.295487   0.944927   30509       9678
2022-01-18   7358       7721  0.301768   0.942505   24383       8192
2022-01-19   6316       6412  0.295818   0.941833   21351       6808
2022-01-20   5982       6059  0.308208   0.927445   19409       6533
2022-01-21   5989       6076  0.307018   0.929621   19507       6536
2022-01-22   7556       7170  0.305800   0.919467   24709       7798
2022-01-23   7435       7266  0.312881   0.925487   23763       7851
2022-01-24   5181       5239  0.321681   0.927749   16106       5647
2022-01-25   4662       4759  0.317835   0.922466   14668       5159
2022-01-26  12613      11008  0.334243   0.911409   37736      12078
2022-01-27   8156       7410  0.325706   0.910319   25041       8140
2022-01-28   7112       6572  0.326268   0.901015   21798       7294
2022-01-29   6757       6487  0.326362   0.902728   20704       7186
2022-01-30   6020       5903  0.321358   0.908573   18733       6497
2022-01-31   3670       3743  0.337595   0.904981   10871       4136
2022-02-01   4029       4423  0.333748   0.905796   12072       4883
2022-02-02   6402       6347  0.323628   0.869929   19782       7296
2022-02-03   5707       5755  0.333489   0.893911   17113       6438
2022-02-04   5490       5596  0.340149   0.894215   16140       6258
2022-02-05   5577       5730  0.340061   0.895592   16400       6398
2022-02-06   5592       5705  0.333532   0.893920   16766       6382
2022-02-07   3559       3720  0.357545   0.904010    9954       4115
2022-02-08   3272       3715  0.351073   0.904774    9320       4106
2022-02-09   5093       5040  0.351363   0.874241   14495       5765
2022-02-10   4581       4792  0.350471   0.892365   13071       5370
2022-02-11   4430       4727  0.345662   0.897304   12816       5268
2022-02-12   4569       4859  0.344077   0.894679   13279       5431
2022-02-13   4548       4650  0.352777   0.887235   12892       5241
2022-02-14   2569       2983  0.372103   0.897413    6904       3324
2022-02-15   2705       3007  0.380771   0.887806    7104       3387
2022-02-16   2870       2712  0.326248   0.867840    8797       3125
2022-02-17   2757       2764  0.343637   0.858652    8023       3219
2022-02-18   2676       2652  0.345781   0.858530    7739       3089
2022-02-19   2973       2976  0.347638   0.867133    8552       3432
2022-02-20   2845       2958  0.354650   0.874372    8022       3383
2022-02-21   2227       2525  0.365742   0.874005    6089       2889
2022-02-22   2184       2415  0.367925   0.878182    5936       2750
2022-02-23   4283       4261  0.338497   0.853737   12653       4991
2022-02-24   3585       3597  0.353690   0.856429   10136       4200
2022-02-25   3274       3751  0.352346   0.870100    9292       4311
2022-02-26   3736       4285  0.363459   0.886613   10279       4833
2022-02-27   3649       4228  0.371513   0.893114    9822       4734
2022-02-28   1980       2350  0.372111   0.880150    5321       2670
2022-03-01   1943       2403  0.365363   0.879254    5318       2733
2022-03-02   3191       3514  0.369842   0.873478    8628       4023
2022-03-03   2555       2958  0.371690   0.881144    6874       3357
2022-03-04   3022       3443  0.381229   0.871866    7927       3949
2022-03-05   3330       3885  0.381793   0.869906    8722       4466
2022-03-06   3039       3640  0.370203   0.873111    8209       4169
2022-03-07   1558       1993  0.373980   0.892122    4166       2234
2022-03-08  

In [204]:
starter_pack_npu = cash_pu.groupBy("date").agg(countDistinct("account_id").alias("pu"), countDistinct(when(col("date") == col("first_mtx_date"), col("account_id"))).alias("npu")).toPandas()

In [214]:
starter_pack_membership_npu = cash_pu.groupBy("date", "membership").agg(countDistinct("account_id").alias("pu"), countDistinct(when(col("date") == col("first_mtx_date"), col("account_id"))).alias("npu")).toPandas()

In [215]:
# starter_pack_npu["npur"] = starter_pack_npu["npu"]/starter_pack_npu["pu"]
starter_pack_membership_npu["npur"] = starter_pack_membership_npu["npu"]/starter_pack_membership_npu["pu"]

In [216]:
# starter_pack_npu.to_csv("./data/starter_pack_npu.csv", index=False)
starter_pack_membership_npu.to_csv("./data/starter_pack_membership_npu.csv", index=False)

In [222]:
starter_pack_npu = pd.read_csv("./data/starter_pack_npu.csv")
starter_pack_membership_npu = pd.read_csv("./data/starter_pack_membership_npu.csv")

In [239]:
starter_pack_membership_npu.pivot_table(values=["pu", "npu", "npur"], columns=["membership"], index="date")

npu                   npur                         pu          \
membership  basic legacy  plus     basic    legacy      plus  basic  legacy   
date                                                                          
2022-01-12  16153  57330  3122  0.949730  0.140809  0.929997  17008  407148   
2022-01-13  16151  31858  2574  0.954946  0.206686  0.874618  16913  154137   
2022-01-14  14321  22406  2193  0.962303  0.233493  0.869893  14882   95960   
2022-01-15  15415  24021  2388  0.954785  0.245325  0.871533  16145   97915   
2022-01-16  10388  13516  1713  0.955043  0.265098  0.858216  10877   50985   
2022-01-17   7908   9015  1226  0.964390  0.295487  0.829499   8200   30509   
2022-01-18   6653   7357  1055  0.965042  0.301727  0.812789   6894   24383   
2022-01-19   5436   6316   965  0.965028  0.295818  0.821277   5633   21351   
2022-01-20   5134   5982   915  0.961783  0.308208  0.765690   5338   19409   
2022-01-21   5149   5989   910  0.959918  0.307018  0.776451   5364   19507   
2022-01-22   6170   7556   988  0.952602  0.305800  0.747918   6477   24709   
2022-01-23   6295   7433   964  0.954945  0.312810  0.765079   6592   23762   
2022-01-24   4474   5181   760  0.963809  0.321681  0.756219   4642   16106   
2022-01-25   4047   4661   709  0.961740  0.317767  0.745531   4208   14668   
2022-01-26   9667  12612  1310  0.961030  0.334217  0.648836  10059   37736   
2022-01-27   6501   8156   892  0.950717  0.325706  0.685100   6838   25041   
2022-01-28   5731   7112   838  0.942909  0.326268  0.689145   6078   21798   
2022-01-29   5596   6756   887  0.942246  0.326314  0.711307   5939   20704   
2022-01-30   5146   6020   749  0.944047  0.321358  0.716061   5451   18733   
2022-01-31   3147   3670   595  0.938003  0.337595  0.761844   3355   10871   
2022-02-01   3783   4029   634  0.938244  0.333748  0.745006   4032   12072   
2022-02-02   5572   6402   767  0.925274  0.323644  0.601569   6022   19781   
2022-02-03   5019   5707   729  0.943067  0.333489  0.653226   5322   17113   
2022-02-04   4958   5489   634  0.937240  0.340108  0.654283   5290   16139   
2022-02-05   5050   5577   673  0.937268  0.340061  0.666337   5388   16400   
2022-02-06   5071   5592   632  0.937685  0.333532  0.648871   5408   16766   
2022-02-07   3267   3559   451  0.942042  0.357545  0.697063   3468    9954   
2022-02-08   3275   3272   439  0.941904  0.351073  0.697933   3477    9320   

                  
membership  plus  
date              
2022-01-12  3357  
2022-01-13  2943  
2022-01-14  2521  
2022-01-15  2740  
2022-01-16  1996  
2022-01-17  1478  
2022-01-18  1298  
2022-01-19  1175  
2022-01-20  1195  
2022-01-21  1172  
2022-01-22  1321  
2022-01-23  1260  
2022-01-24  1005  
2022-01-25   951  
2022-01-26  2019  
2022-01-27  1302  
2022-01-28  1216  
2022-01-29  1247  
2022-01-30  1046  
2022-01-31   781  
2022-02-01   851  
2022-02-02  1275  
2022-02-03  1116  
2022-02-04   969  
2022-02-05  1010  
2022-02-06   974  
2022-02-07   647  
2022-02-08   629

In [229]:
starter_pack_membership_npu[starter_pack_membership_npu.membership == "legacy"]

,date,membership,pu,npu,npur
6,2022-01-28,legacy,21798,7112,0.326268
7,2022-01-25,legacy,14668,4661,0.317767
11,2022-01-29,legacy,20704,6756,0.326314
14,2022-02-07,legacy,9954,3559,0.357545
17,2022-02-04,legacy,16139,5489,0.340108
18,2022-01-23,legacy,23762,7433,0.312810
25,2022-01-13,legacy,154137,31858,0.206686
27,2022-01-27,legacy,25041,8156,0.325706
28,2022-01-26,legacy,37736,12612,0.334217
29,2022-01-22,legacy,24709,7556,0.305800


In [230]:
starter_pack_membership_npu[starter_pack_membership_npu.membership == "basic"]

,date,membership,pu,npu,npur
1,2022-01-22,basic,6477,6170,0.952602
12,2022-01-29,basic,5939,5596,0.942246
13,2022-02-05,basic,5388,5050,0.937268
15,2022-02-06,basic,5408,5071,0.937685
16,2022-01-31,basic,3355,3147,0.938003
19,2022-01-24,basic,4642,4474,0.963809
23,2022-01-26,basic,10059,9667,0.961030
26,2022-01-28,basic,6078,5731,0.942909
31,2022-01-16,basic,10877,10388,0.955043
39,2022-01-18,basic,6894,6653,0.965042


In [231]:
starter_pack_membership_npu[starter_pack_membership_npu.membership == "plus"]

,date,membership,pu,npu,npur
0,2022-01-30,plus,1046,749,0.716061
2,2022-02-07,plus,647,451,0.697063
3,2022-01-18,plus,1298,1055,0.812789
4,2022-02-02,plus,1275,767,0.601569
5,2022-01-21,plus,1172,910,0.776451
8,2022-01-31,plus,781,595,0.761844
9,2022-01-26,plus,2019,1310,0.648836
10,2022-01-13,plus,2943,2574,0.874618
20,2022-01-17,plus,1478,1226,0.829499
21,2022-02-03,plus,1116,729,0.653226
